In [1]:
import pandas as pd
import numpy as np
import os
import re

from pathlib import Path
import gdt

from Bio import Entrez

In [2]:
nth_iteration = 1

In [ ]:
# Defines all the global variables used in the script.
# Change these variables to match your local setup.
# The most_recent_gdt_file variable should be set to the path of the most recent GDT file,
# OR the stripped GDT file used in filter command, if applicable.

DATA_DIR = "../test/Test_group16"
AN_missing_dbxref = "../test/Test_group16/AN_missing_dbxref"
#most_recent_gdt_file = "../test/Test_group16/Test_group16.gdt"
remove_orfs = True
organelle_type = "MT"
gff_suffix = ".gff3"

Entrez.email = 'dupin@alunos.utfpr.edu.br'
Entrez.api_key = 'b3abc1ac7ae9ac035af84ec1abf895878d09'

In [4]:
most_recent_gdt_file = ""

In [5]:
# Check if all variables exist
DATA_DIR = Path(DATA_DIR).resolve()
AN_missing_dbxref = Path(AN_missing_dbxref).resolve()

if not DATA_DIR.exists() and not DATA_DIR.is_dir():
    raise FileNotFoundError(f"Data directory {DATA_DIR} does not exist or is not a directory.")

if not AN_missing_dbxref.exists() and not AN_missing_dbxref.is_file():
    raise FileNotFoundError(f"AN missing dbxref {AN_missing_dbxref} does not exist or is not a file.")

if not most_recent_gdt_file:
    if nth_iteration > 1:
        raise FileNotFoundError(f"Most recent GDT file {most_recent_gdt_file} does not exist or is not a file.")
    else:
        print(f"If you set up a stripped GDT file, please set the path to it in the most_recent_gdt_file variable.")
        print(f"Otherwise, ignore this message.")
else:
    most_recent_gdt_file = Path(most_recent_gdt_file).resolve()
    if not most_recent_gdt_file.exists() and not most_recent_gdt_file.is_file():
        raise FileNotFoundError(f"Most recent GDT file {most_recent_gdt_file} does not exist or is not a file.")

If you set up a stripped GDT file, please set the path to it in the most_recent_gdt_file variable.
Otherwise, ignore this message.


In [6]:
_, logger = gdt.logger_setup.logger_creater(log_file=DATA_DIR / '0_test_2.log', console_level="DEBUG", file_level="TRACE")
logger.debug("Running from notebook AN_missing_dbxref")

2025-05-21 16:08:43,154 - DEBUG - Logger setup complete. Logging to /home/brenodupin/matheus/gdt/test/Test_group16/0_test_2.log
2025-05-21 16:08:43,158 - DEBUG - Running from notebook AN_missing_dbxref


In [7]:
with open(AN_missing_dbxref, "r") as f:
    ANs = [line.strip() for line in f.readlines() if line.strip()]
print(f"len(ANs): {len(ANs)}")

len(ANs): 118


In [8]:
missing_dbxref_path = DATA_DIR / "missing_dbxref"
missing_dbxref_path.mkdir(exist_ok=True)

In [9]:
# Load the GDT file
if most_recent_gdt_file:
    gene_dict = gdt.gene_dict.create_gene_dict(most_recent_gdt_file, max_an_sources=0)
else:
    gene_dict = {}

temp_gene_dict = {}

In [10]:
temp_list = []
for AN in ANs:
    an_path = DATA_DIR / f'{AN}{gff_suffix}'
    if not an_path.exists():
        logger.error(f'Error: {AN} does not exist (an_path: {an_path})')
        continue
    
    df = gdt.gff3_utils.load_gff3(an_path, query_string='type == "gene"', usecols=['seqid', 'start', 'end', 'type', 'attributes'])
    df = gdt.gff3_utils.filter_orfs(df) if remove_orfs else df

    df['gene_id'] = df['attributes'].str.split(';').str[0].str.replace('ID=', '', regex=False)
    df['in_gene_dict'] = df['gene_id'].isin(gene_dict)
    df_missing = df[~df['in_gene_dict']].copy()

    temp_list.extend(df_missing[['gene_id', 'seqid']].to_dict('records'))

In [11]:
agg_dbxref = (pd.DataFrame(temp_list).groupby('gene_id')['seqid']
                .agg(list)
                .sort_index())  # Sort by gene_id

In [ ]:
# Write here anything you want to add to the missing_dbxref file, or leave it empty
comment = "manual insertion from missing_dbxref_compiled"

In [ ]:
with open(missing_dbxref_path / 'missing_dbxref_compiled.txt', 'w+') as f1:
    for gene_id, seqid in agg_dbxref.items():
        f1.write(f'{gene_id} #gn {" ".join(seqid)}{ " #c " + comment if comment else "" }\n')

After manual parsing of missing_dbxref_compiled.txt,  
create missing_dbxref_problems.txt, with names that  
are not readily indentifiable or that need deeper investigation.

### Deeper investigation using other gff attributes, primarily 'Name='

In [26]:
an_with_no_dbxref = set()
with open(missing_dbxref_path / 'missing_dbxref_problems.txt', 'r') as f:
    for line in f:
        if line.startswith('#') or not line.strip():
            continue
        # Get ANs part (after '||') and split into individual ANs
        if '#c' in line:
            line = line.split('#c')[0].strip()
        
        ans = line.split('#gn')[1].strip().split()
        # Add each AN to the set
        an_with_no_dbxref.update(ans)

In [27]:
an_with_no_dbxref

{'AP012272.1', 'LC659289.1', 'MH725795.1'}

In [68]:
temp_list = []
for AN in an_with_no_dbxref:
    an_path = DATA_DIR / f'{AN}{gff_suffix}'
    df = gdt.gff3_utils.load_gff3(an_path, query_string='type == "gene"', usecols=gdt.GFF3_COLUMNS) # TODO change query_string!
    df = gdt.gff3_utils.filter_orfs(df) if remove_orfs else df


    df['gene_id'] = df['attributes'].str[3:].str.partition(';', expand=False).str[0]
    df = df[~df['gene_id'].isin(gene_dict)]
    
    df['source_gid'] = 'name'
    df['name'] = df['attributes'].str.extract(r'Name=([^;]*)(?:;|$)')

    # if 'name' is NaN, try to extract 'description='
    # change source to 'description' if 'description=' is found
    df['source_gid'] = np.where(df['name'].isna(), 'description', df['source_gid'])
    df['name'] = df['name'].fillna(df['attributes'].str.extract(r'description=([^;]*)(?:;|$)', expand=False))

    # if 'name' is still NaN, print a warning
    if df['name'].isna().any():
        print(f'Warning: {AN} has NaN names')
        df['source_gid'] = np.where(df['name'].isna(), 'gene_synonym', df['source_gid'])
        df['name'] = df['name'].fillna(df['attributes'].str.extract(r'gene_synonym=([^;]*)(?:;|$)', expand=False))

        df['source_gid'] = np.where(df['name'].isna(), 'stil_nan', df['source_gid'])
        print(df[df['name'].isna()])

    temp_list.extend(df[['name', 'gene_id', 'seqid', 'source_gid']].to_dict('records'))

gene_ans_name_df = pd.DataFrame(temp_list)

In [ ]:
add_gdt_compliance = True
comment = 'Manual from missing_dbxref_names_raw'

In [ ]:
if add_gdt_compliance:
    gdt_str = f' #gd MANUAL{ " #c " + comment if comment else "" }'
else:
    gdt_str = ""
    
with open(missing_dbxref_path / 'missing_dbxref_names_raw.txt', 'w+') as f1:
    for name in sorted(gene_ans_name_df['name'].unique()):
        f1.write(f'{name}{gdt_str}\n')

The user must now parse missing_dbxref_names_raw.txt into two files:  
missing_dbxref_names_clean.txt  
missing_dbxref_names_unk.txt  
  
missing_dbxref_names_clean.txt should contain all easily identifiable gene names,  
all the names __must__ also be in your current GDT version, because the next step  
will automaticaly add all gene_ids that have names inside missing_dbxref_names_clean.txt.  
  
missing_dbxref_names_unk.txt should contain all the names that you dont know or you're not certain about,  
the next step for theses names are matching feature using CDS, then extracting information about these features,  
to better classify the original gene.

In [78]:
# missing_dbxref_names_clean.txt

# Check if the names exist in the gene_dict
names = set()
with open(missing_dbxref_path / 'missing_dbxref_names_clean.txt', 'r') as f1:
    for line in f1:
        line = line.strip()
        if not line:
            continue

        if '#gd' in line:
            line = line.split('#gd')[0].strip()
        
        names.add(line)


In [ ]:
# TODO check where is being read gene_dict
for name in names:
    if name not in gene_dict:
        print(f"W: {name} not in gene_dict")

In [ ]:
# TODO automatically add the gene_id to the gene_dict that have a name in the missing_dbxref_names_clean.txt file

In [ ]:
# missing_dbxref_names_unk.txt

In [ ]:
def print_df_rows(cds_trna):
    for row_cds in cds_trna.itertuples():
        print(f'\ts: {row_cds.start}| e: {row_cds.end} | {row_cds.attributes}')

In [ ]:
temp_gene_an = {}
an_set = set()
with open(f'{folder}_dbxref_names_unk.txt', 'r') as f:
    for line in f:
        raw = line.strip().split('|')[1].strip()
        gene_id, seqid = raw.split(' # ')
        if ' ' in seqid:
            seqid = [x.strip() for x in seqid.split()]
        else:
            seqid = [seqid.strip()]
        gene_id = gene_id.strip()
        
        an_set.update(seqid)
        temp_gene_an[gene_id] = seqid

In [ ]:
dict_an_gene = {}
for an in an_set:
    for gene_id, seqid in temp_gene_an.items():
        if an in seqid:
            if an not in dict_an_gene:
                dict_an_gene[an] = []
            dict_an_gene[an].append(gene_id)

In [ ]:
dict_an_gene
